<a href="https://colab.research.google.com/github/LukasEder1/CKE-Evaluation/blob/master/CreateKeywordCollections.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone "https://github.com/LukasEder1/CKE-Evaluation.git"
%cd "/content/CKE-Evaluation"

Cloning into 'CKE-Evaluation'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (49/49), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 49 (delta 15), reused 14 (delta 0), pack-reused 0
Unpacking objects: 100% (49/49), 228.72 KiB | 1.62 MiB/s, done.
/content/CKE-Evaluation


In [ ]:
%pip install -r requirements.txt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/boudinfl/pke.git to /tmp/pip-req-build-rs_ciwxd
  Running command git clone --filter=blob:none --quiet https://github.com/boudinfl/pke.git /tmp/pip-req-build-rs_ciwxd
  Resolved https://github.com/boudinfl/pke.git to commit 8f1d05dcc52041c9920ba0f9d5231fe6086d12c4
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.3/97.3 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 68.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 12.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.2/60.2 KB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━

# Keyword Collection Creation

In [ ]:
from contrastive_keyword_extraction import contrastive_extraction, final_score
import sqlite3
import pandas as pd
from baselines import *
from tqdm import tqdm
import string
import pickle
import sentence_comparision
import sentence_importance
import summary
import utilities
import keyword_extraction
import nltk

nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
with open("policy_data.pkl", "rb") as file:
  policies = pickle.load(file)

In [ ]:
def create_collection(sites, 
                      ke_extractor = keyword_extraction.extract_yake, 
                      num_keywords=10,
                      max_ngram=2, 
                      sentence_matcher = sentence_comparision.match_sentences_semantic_search,
                      importance_estimator = sentence_importance.text_rank_importance,
                      name_prefix="",
                      make_data_persistent=False, 
                      path="dataframes",
                      threshold=0.6,
                      stopwords=nltk.corpus.stopwords.words("english"),
                      combinator=utilities.alpha_combination,
                      gamma = 0.5,
                      num_splits=1,
                      matching_model="all-MiniLM-L6-v2"):
    
    for i in tqdm(sites.keys()):

        documents = sites[i]
        
        # run CKE-pipeline
        # Extract Keywords, and Matched sentences
        keywords, matched_dict, changed_indices, additions, deletions, new_indices, ranking, removed, matched_indices, unified_delitions = contrastive_extraction(documents, 
                                                                            max_ngram=max_ngram,
                                                                            min_ngram=1, 
                                                                            show_changes=False, 
                                                                            symbols_to_remove=string.punctuation,
                                                                            importance_estimator=importance_estimator,
                                                                            match_sentences=sentence_matcher,
                                                                            threshold=threshold,
                                                                            extra_stopwords=stopwords,
                                                                            top_k=int(num_splits),
                                                                            combinator=combinator,
                                                                            alpha_gamma=gamma,
                                                                            matching_model=matching_model)
        
        

    
        # create itermediate
        kws, scores = extract_from_dict(keywords)
        
        pipeline_frame = pd.DataFrame({'keyword': kws, 'score': scores})
        
        #extractor = lambda x: ke_extractor(x, max_ngram_size=max_ngram, numOfKeywords=num_keywords)
        
        # create CKE on the specified baseline
        baseline_keywords = baseline_diff_content(additions, unified_delitions, ke_extractor, num_keywords, max_ngram)
        
        baseline_kws, baseline_scores = extract_from_tuple_list(baseline_keywords)
        
        baseline_frame1 = pd.DataFrame({'keyword': baseline_kws, 'score': baseline_scores})
        
        
        # create CKE for baseline method 2
        baseline_keywords2 = baseline_keywords_in_diff(documents, ke_extractor, additions, deletions, candidates=50, max_ngram=max_ngram)
        
        baseline_kws2, baseline_scores2 = extract_from_dict(baseline_keywords2)
        
        baseline_frame2 = pd.DataFrame({'keyword': baseline_kws2, 'score': baseline_scores2})
        
        # Baseline 3
        baseline_keywords3 = baseline3(documents, additions, unified_delitions, max_ngram)
        
        baseline_kws3, baseline_scores3 = extract_from_dict(baseline_keywords3)
        
        baseline_frame3 = pd.DataFrame({'keyword': baseline_kws3, 'score': baseline_scores3})

        # Baseline 4
        baseline_keywords4 = baseline4(documents, max_ngram, stopwords)
        
        baseline_kws4, baseline_scores4 = extract_from_dict(baseline_keywords4)
        
        baseline_frame4 = pd.DataFrame({'keyword': baseline_kws4, 'score': baseline_scores4})

        # decide, wether to actually save the data
        if make_data_persistent:
            
            pipeline_frame.to_csv(f"{path}/{name_prefix}_pipeline_keywords_{i}.csv", index=False)
            
            baseline_frame1.to_csv(f"{path}/{name_prefix}_baseline1_keywords_{i}.csv", index=False)
            
            baseline_frame2.to_csv(f"{path}/{name_prefix}_baseline2_keywords_{i}.csv", index=False)
            
            baseline_frame3.to_csv(f"{path}/{name_prefix}_baseline3_keywords_{i}.csv", index=False)
            
            baseline_frame4.to_csv(f"{path}/{name_prefix}_baseline4_keywords_{i}.csv", index=False)
    
    return pipeline_frame, baseline_frame1, baseline_frame2, baseline_frame3, baseline_frame4

In [ ]:
a = create_collection(policies, 
                  ke_extractor = keyword_extraction.extract_yake, 
                  num_keywords=15, # relevant for baselines
                  max_ngram=2, 
                  sentence_matcher = sentence_comparision.match_sentences_semantic_search,
                  importance_estimator = sentence_importance.text_rank_importance,
                  name_prefix="standard",
                  make_data_persistent=True, 
                  path="Dataframes",
                  threshold=0.6,
                  stopwords=nltk.corpus.stopwords.words("english"),
                  combinator=utilities.alpha_combination,
                  gamma = 0.5,
                  num_splits=2,
                  matching_model ="msmarco-distilbert-base-v4")

100%|██████████| 25/25 [04:17<00:00, 10.31s/it]


(            keyword     score
 0       information  0.017896
 1               may  0.013724
 2          services  0.013484
 3                pr  0.012073
 4               use  0.010210
 ..              ...       ...
 856  collection use  0.000377
 857        use data  0.000377
 858        data use  0.000377
 859         use prn  0.000377
 860      prn mobile  0.000377
 
 [861 rows x 2 columns], Empty DataFrame
 Columns: [keyword, score]
 Index: [], Empty DataFrame
 Columns: [keyword, score]
 Index: [], Empty DataFrame
 Columns: [keyword, score]
 Index: [],                       keyword     score
 0                 information  0.023857
 1                         com  0.014911
 2              prnewswire com  0.010934
 3                          us  0.010934
 4                          pr  0.010934
 ..                        ...       ...
 638                        12  0.000994
 639                 relations  0.000994
 640           policy explains  0.000994
 641  information distribut

In [ ]:
from itertools import product


def cartesian_product(params):
    
    # gett all possible combinations
    return list(product(*params.values()))



# Parameters to Use

In [ ]:
parameters = {"matcher": [sentence_comparision.match_sentences_semantic_search,
                          sentence_comparision.match_sentences_tfidf_weighted],
              
             "ie": [sentence_importance.text_rank_importance],
              
             "threshold": [0.7]
              
             "matching-model": []
             }


In [ ]:
combinations = cartesian_product(parameters)
num_of_combinations = len(combinations)

In [ ]:
def create_cartesian_collection(params, 
                                sites, 
                                baseline_ke_extractor = keyword_extraction.extract_yake, 
                                num_keywords=10, 
                                max_ngram=2, 
                                make_data_persistent=False,
                                file_prefix = "combination",
                                path="Dataframes",
                                stopwords=nltk.corpus.stopwords.words("english"),
                                combinator=utilities.alpha_combination,
                                gamma = 0.5,
                                num_splits=1,
                                matching_model="all-MiniLM-L6-v2"):
            
        
    combinations = cartesian_product(params)
    
    number_of_combinations = len(combinations)
    
    count = 0
    
    for combination in combinations:
        
        matcher, ie, threshold = combination
        
        print(f"Contrastive Keyword Extraction pipeline is being ran with combination {count}:")
        
        pipeline_frame, baseline_frame1, baseline_frame2, baseline_frame3, baseline_frame4 = create_collection(sites = sites, 
                                                                  ke_extractor = baseline_ke_extractor, 
                                                                  num_keywords = num_keywords, 
                                                                  max_ngram = max_ngram, 
                                                                  sentence_matcher = matcher,
                                                                  importance_estimator = ie,
                                                                  name_prefix=f"{file_prefix}_{count}",
                                                                  make_data_persistent=make_data_persistent,
                                                                  path=path,
                                                                  threshold=threshold,
                                                                  combinator=combinator,
                                                                  gamma = gamma,
                                                                  num_splits=num_splits,
                                                                  matching_model = matching_model)




        count += 1
        

In [ ]:
create_cartesian_collection(parameters, 
                            policies, 
                            baseline_ke_extractor = keyword_extraction.extract_yake, 
                            num_keywords=15, 
                            max_ngram=2, 
                            make_data_persistent=True,
                            file_prefix = "combination",
                            path="Dataframes",
                            stopwords=nltk.corpus.stopwords.words("english"),
                            combinator=utilities.alpha_combination,
                            gamma = 0.5,
                            num_splits=1,
                            matching_model="all-MiniLM-L6-v2")

Contrastive Keyword Extraction pipeline is being ran with combination 0:


100%|██████████| 25/25 [03:57<00:00,  9.49s/it]


Contrastive Keyword Extraction pipeline is being ran with combination 1:


100%|██████████| 25/25 [04:17<00:00, 10.30s/it]


# Evaluate Results


## Compare against baselines

In [ ]:
summary.extensive_summary(list(policies.keys()),
                          show_results=True, 
                          k=15,
                          name_a = f"standard_pipeline_keywords", 
                          name_b = f"standard_baseline1_keywords",
                          save_prefix=f"standard_", 
                          path='Dataframes')


,Site,F1,Precision,Recall,IoU,#overlaps
0,90536,0.0,0.0,0.0,0.0,0
1,90344,0.0,0.0,0.0,0.0,0
2,98640,0.0,0.0,0.0,0.0,0
3,98585,0.0,0.0,0.0,0.0,0
4,99880,0.0,0.0,0.0,0.0,0
5,108079,0.0,0.0,0.0,0.0,0
6,90555,0.0,0.0,0.0,0.0,0
7,90545,0.0,0.0,0.0,0.0,0
8,98553,0.0,0.0,0.0,0.0,0
9,98572,0.0,0.0,0.0,0.0,0


## Manual Policy Keywords

In [ ]:
manual = {

    "90536": [
            "European Union", "cookies", "bugs",
            "personally identifiable", "Salesforce",
            "identifiable information", "customer testimonial",
            "version", "Third Party", "clear gifs", "web beacons",
            "service providers",  "blog"],


    "90344": ["Telerik", "May", "Progress Software", "software products", "software websites",
         "community progress", "Telerik Analytics", "please note", "Third party"
        "service providers", "geo-location",
         "google Analytics",  "children"],


    "98640": ["validation page", "scripts", "removing HTML5", "tools removing", "management tools",
         "various browsers", "clear gifs", "transparency", "may offer", "cookies", "flash cookies",
         "browsers may"],


    "98585": ["account settings", "center cloud", "cloud console", "security center", "console",
          "web beacons", "kaspersky securtiy", "personal data",
           "data", "email address", "solution kaspersky", "documentation section", "account registration"],


    "99880": ["reproductive science", "science center", "Michele Abbott",
    "electronic", "health information", "insurance company", "health", "shady grove",
    "treatment"], #meta: shady grove

    "108079": ["destination nsw","Insights", "Viva Sydney","Sydney Club", "130000 visitors","visitors per", "per month",
    "e-newsletter", "personal information", "please contact"],

    "90555":  ["serve ads", "DART cookie", "registration number", "privacy policy", "data control", "data protection"],


    "90545":  ["Yahoo privacy", "Privacy Policy", "AOL", "Oath", "AOL registred","registred users" ,"Verizon","oath privacy"],

    "98553": ["customer" "affiliated websites", "providers", "billing information",
    "contact us", "service plans", "partners", "advertising partners", "march 8", "april 20"],


    "98572": ["job applications", "Privacy Shield", "Shield Framework",
          "secure data", "data recovery", "recovery services" "personal data"],

    "98706": ["Australian privacy", "post", "contact us", "telephone", "SEEK Learning", "SEEK commerical"
        "Head office"],


    "108052":  ["Reloaded Games", "K2 network"],


    "108097": ["Website Security", "PKI solutions", "DigiCert Inc" "Symantec website",  "Global company",
            "data transfer","transfers", "Download privacy", "service providers", "contractors", "around the world",
            "privacy statement", "privacy policy"],


    100541: ["live", "browse history", "Synchrony Bank", "Synchrony", "GE money", "money bank", "GEMB",
         "Credit Card", "bank Credit", "Billing Act", "EVINE live", "online privacy"],

    108771: ["tap", "ALTERNATIVE PRESS", "alternative", "press llc"],

    2435: ["properties", "sites", "requests", "california", "545", "5215", "freeway suite",
        "john carpenter" ,"700", "1400", "Connor", "Blvd", "Connor Blvd"],

    100595: ["evine", "shopnbc", "ge money", "synchrony", "mobile", "mobile device", "money bank", "Synchrony Bank",
         "california", "credit card", "evine card", "ShopNBC platinum", "ShopNBC platinum rewards", "MasterCard"],

    108778: ["DART", "DART cookie", "parties", "third parties", "email", "log file", "file information",
        "personally identifiable"], # email is written differently

    100419: ["toxic shock", "tv", "shock tv" "advertising companies", "advertising", "third parties", "DART cookie",
         "web site", "serve ads", "web beacons", "cookie", "data collection"],



    108438: ["TakeLessons Live", "TakeLessons", "Released Subject", "Subject matter",
     "profile", "personal information", "current version", "update", "privacy policy",
     "unsubscribe"],

    108835: ["personally identifiable", " identifiable information", "cookies", "use", "privacy statement",
    "persitent cookies", "credit card", "support@tarot.com", "site"],#meta: site

    # small change examples
    106348: ["advertising", "www", "third party", "ad network", "strayer university"],
    90041: ["certain services", "mobile app", "app", "pages listed"],

# BEW
  89460: ["children", "player privacy", "privacy policy", "european union",
          "commerical partner", "personal information", "review"], # personal information (on a meta level)

  89884: ["debit card", "details", "credit debit", "SecureTrading", "order products", "competion terms",
      "conditions"],

  106296: ["Family mobile", "walmart", "geolocation", "phone" "device", "california",
         "network", "track", "data", "personal information"],

  89510: ["google adsense", "third party", "party advertisers","cookie", "dart cookie",
   "advertisers", "advertising partners", "partners include"],

  97991: ["google analytics", "version", "google", "GDPR",
        "screenpresso application", "amazon web", "web service", "Ip address"],

  89035: ["notice", "policy" "device", "automatically", "location information",
      "personally identifiable", "identifiable information", "service providers",
      "CareerBuilder", "Adobe Flash"], # advertising on a meta level

  98022: ["employees and contractors","Audio/Video Transcription", "Transcription Service",
         "required sign", "sign non", "non disclousure", "disclousure agreement"],

  # ~15 Min break: return at 14:35

  89139: ["location information", "device", "personally identifiable", "identifiable information",
   "automatically collect", "third parties", "service providers", "Last updated", "opt out"],

  89648: ["can-spam compliance",  "marketing act", "securtiy measures", "nonsolicited pornography",
        "security measures", "can-spam compliance", "controlling assault"],

  97653 : ["secure", "personal information", "mobile", "data", "technical", "opt", "opt out", "improve"],

  98103: ["swingers", "free trial", "customer service", "lifesytle website", "travel", "local swingers",
         "find local", "membership", "lifestyle events"], # SDC on a meta level

  89170: ["cookies", "utilize cookies", "RTM websites", "third-party", "web browser",
   "advertising", "ad servers", "detailed information", "ip address", "data collection"],

  89763: ["cookies", "com", "mobile devices", "safe harbor", "email address"
      "google",  "advertising", "priceline", "please click"], # Meta Level: data, cookies, advertising

  97732: ["ScienceForums.net staff", "policy", "notice", "servers", "log files",
            "IP addresses", "web servers", "explicit consent", "action"],

  98116: ["LookupRealty", "SDLookup"],

  89273: ["healthkit database", "planned parenthood", "Donor Services" "parenthood National",
    "national office", "Apple's Healthkit", "ppfa", "online services"],# Meta: parenthood, planned parenthood

  89768: ["pricena fz",  "third parties", "right holder", "rights reserved", "updated", "pricena.com acts"], # meta level: pricena

  97787: ["dataprivacy aip.org","practices", "advertising based", "peer review", "personal information", "aip publishing"], # meta: personal information

  89306: ["dart cookie", "serve ads", "power point", "cookie", "third party", "vendor", "ppst", "google"],

  89860: ["cookies","primo website", "privacy statement", "revisions", "cookies disabled", "disabled flagged",
  "recognize browser"],

  97799: ["Digital offering", "personal data", "disclose", "international privacy", "privacy policy", "rights",
    "web sites", "US privacy"], # Meta: Digital offering

  # BREAK: 17:00

  # Restart: 19:00

  # TODO: 13 <- Mi 19:30
  104276: ["ad network", "advertising", "network", "network provider", "advertising notice"
          "advertisements", "site", "sending duplicative", "duplicative advertisements", "web beacons",
          ],

  80227: ["nokta Internet", "personal data", "data", "personal", "data processing",
        "protection act", "processed", "act", "Flash Player"],

  96434: ["CustomerNation", "privacy practices", "customer Service",
        "tom clinkscales", "web site", "address"],

  96759: ["gdpr", "New Zealand", "Swiss", "personal information",
      "personal information", "sessions clicks",  "visits sessions",
      "third party", "credit card", "google analytics", "user analytics"],

  104303: ["third-party vendors", "internet", "questions", "cookies", "opt out",
            "visitors", "Remarketing", "vendors", "SPC", "visiting", "website tracking",
            "information share"], # META: spc


  96461: ["IP addresses", "collect", "Herald & Times",  "Newsquest Herald",
          "personally identifiable", "identifiable information", "web beacons",
          "aggregate", "CV Database", "nonpersonally identifiable",  "email client"],# meta: collect, information, aggregate, identifiable

  # Finish 20:15


  #TODO (DO): 27 #NOTE: Done 16
  # START 10:30 <- left overs
  96970: ["email", "cookies", "use cookies", "privacy", "personal information",
        "information", "third party", "thirdparty partner", "children privacy",
        "email policy"], # meta: personal information, information

  104311: ["Github", "ordered list", "orderedlist", "support",  "inc"],

  96180: ["Australian", "disclose", "consent", "information", "members", "sponsors"], #skip

  96505: ["safe harbor", "shield framework", "privacy shield", "environmental variables",
        "Eu", "HTML5", "single sign", "swiss safe", "third party", "personal data"
        "facebook connect", "saba software"],#Meta: cookies

  96971: ["www", "secure shopping", "credit card", "personal information",
        "privacy policy", "cookies", "information", "children privacy"],#meta: www, information

  #31161: ["outside", "United states", "telemarketing", "Switzerland",
 #        "california", "transaction", "nevada", "applicable law", "protected"], # meta:applicable law

  # Finish: 11:00
  #NOTE: 10 min break

    #NOTE: Start 11:11
    96240: ["daily beast", "beast implemented", "recaptcha technology", "google invisible", "invisible recaptcha", "www", "http",
      "Nielsen",], #Meta: www, http

    96506: ["social media", "Widgets", "media widgets", "media Features",
        "ip address", "contact us", "third party"],

    97065: ["TRUSTe consent", "consent manager", "Google", "analytics", "Google analytics", "user id",
        "Adobe target", "Ip addresses", "sap group", "Adboe analytics", "Russian Federation"],#meta: SAP, user

    31168: ["Security", "opens", "new window", "collected information", "preferences", "Introduction",
    "customers", "fca US"],# site-buttons: opens, new windows # headers:fca, fca us

    96325: ["data", "rights", "data", "email protected",
    "general data", "data protection", "protection regulation", "European union"],#meta processing, data

    96665: ["personal information", "cookies", "browser", "credit card", "cutomers", "operate stores",
         "affiliated businesses", "Web sites", "please click", "SAFRA"],

  96346: ["google analytics", "pii", "web browser", "browser", "privacy policy", "analytics privacy"],


  80219: ["Siemens", "Solutions", "account", "google", "email address", "use cookies",
        "google analytics", "nokia Solutions", "site content", "enforce"],

  96414: ["ryan homes", "remains strictly", "strictly anonymous", "third party", "homes may",
         "information received", "internal statistical", "statistical purposes",
         "partner"],#meta: ryan homes # mentioned more often: strictly anonymous

  96711: ["privacy policy", "sainsbury bank", "adobe acrobat", "contact us",
         "top", "send us", "email", "explicit consent", "privacy statement"], # meta: privacy policy

  # #NOTE: 14:40
  102943: ["DotNetNuke", "DNN CORP"],

  104225: ["privacy policy", "google analytics", "device identifiers", "mobile device",
      "com", "advertiser feature", "analytics demographics", "identifiable information"],
      #meta: privacy policy, identifiable information

  94843: ["advertising", "third party", "services", "interest based",
      "behavioral advertising", "tracking analytics", "opt out"], # meta: third party, services,

  95640: ["Roland Berger", "countries outside", "eu eea", "third parties", "use data",
      "data protection", "Berger Strategy"], #Meta: Roland Berger, Berger Strategy

  103341: ["sensitive information", "Virtual terminal", "user", "Customer service",
      "take precautions", "online transactions",],

  104249: ["particle", "spark"],

  94977: ["Tigrent Learning", "Legacy Education", "education alliance", "alliance inc",
  "tel 2395420643", "Trowsers Road", "Parkshot House"],

  103581: ["SolarWinds Website", "statement", "privacy statement", "third party", "use cookies"], # meta: SolarWinds Website

  104254: ["Sparked", "Extraordinaries", "customers", "volunteer user", "phishing", "personal data",
  "client"],

  95008: ["fraud prevention", "prevention agencies", "details", "personal information",
    "illegal transactions", "failed attempts"],

  # Break

  95819: ["donation service", "advertising", "identifiable information", "internal businesses",
      "id address", "AppChoices"],

  103865: ["back top", "top", "Sony centre", "Sony styles", "third parties",
    "record information", "3ps", "style"], # meta: top/back

  104257: ["google analytics", "social media", "third party", "credit card", "email address",
      "service", "sparkfun electronics", "law enforcement", "60 days", "servers", "track"],

  104259: ["privacy shield", "standard contractual", "contractual clauses", "shark hire",
        "Google Adwords", "third parties", "Data protection", "data processing", "data",
        "agreement", "Controller"], # meta: shark hire, data


  95866: ["CTI", "PII", "web beacons", "cookies", "third parties", "parents",
      "surveys", "Other Entities", "account", "credit card"],


    103892: ["sca websites", "sony electronics", "sca", "SEL", "new york",
        "third party", "Device", "promotions", "service providers", "Sony drive"],
  # BREAK 10 more


  #95323: ["time warner", "warner cable", "spectrum", "tracking tools", ""],

  95888: ["anonymous data", "back top", "Rovi property", "data", "personal data",
       "public content", "Europe", "Social Networking", "Delaware", "data protection", "england"],

  103927: ["Columbia Sportswear", "Sportswear Company", "Safe Harbor", "harbor framework", "united states",
            "Switzerland", "Swiss"],


  104127: ["Tsogo sun", "information", "personal information", "personally identifiable",
            "identifiable information", "google analytics", "cookies", "Display advertising"], # meta: Tsogo Sun

  94812: ["third party", "web beacons", "social media", "media features",
        "Ip address", "Pixel tags", "tracking technologies", "may use"], #meta: tracking technologies

  95516: ["personal information", "non-personal information", "Rockford fosgate",
       "may collect", "information", "Ip address"],

  104129: ["children", "from outside", "united states", "third parties", "tailored advertising",
      "privacy setting", "may receive", "receive tailored"],

  94836: ["web sites", "revenue science", "testimonials", "personally identifiable",
      "identifiable information", "information practices"], #meta: web sites, revenue science

   102375: ["Boulder brands", "Smart Balance", "cosumermail", "webmaster"],

   4072: ["functionality cookies", "performance cookies"],
   4293: ["cookies", "additional information", "third party", "google analytics",
            "use cookies", "information", "website", "User Experience"],

   102421: ["safe harbor", "enterprise", "enterprise users", "enterprise level",
      "Harbor framework", "harbor privacy", "privacy principles", "Customer Service",
      "Test Question", "profile page", "profile information"],

   102542: ["Mercedes Me", "owners online", "such cases", "cases may", "may employ","employ outside",
    "outside research", "research company", "company call", "call behalf"], # 1 new sentence <- keywords in 2-grams

   4278: ["social security", "link", "external link", "session cookies", "Air force",
            "monitoring"], # meta: air force
  # 6 MORE:

    93866: ["RedZee", "Link Positions", "search results", "identify"],

    93873: ["personally identifiable", "identifiable information", "affiliated companies", "real estate",
        "vendors", "broker"],

    93932: ["ReferralSaaSquatch", "Yupiq", "personal data", "data", "personal information", "com"],

    94137: ["phone - call", "call tollfree", "back top",  "personal information", "non-personal", "products and services"],

    93880: ["reed jewelers", "sites", "comentity"],

    94002 : ["Social media", "widgets", "persitent", "web site", "third party", "session id",
    "media features", "Facebook Connect"]
}

## Compare Against Manually Annotated Data

In [ ]:
import string
import nltk

def remove_punctuation(text, symbols=string.punctuation):
    return "".join([char for char in text if char not in symbols])

stop_words = nltk.corpus.stopwords.words("english")

In [ ]:
def process_text(string):
    string = remove_punctuation(string)
    words_to_use = []
    for word in nltk.word_tokenize(string):
        if word not in stop_words:
            words_to_use.append(word)
        
    return " ".join(words_to_use)

In [ ]:
manual = {site: [process_text(word) for word in words] for site, words in manual.items()}

In [ ]:
def evaluate_manual(manual, compare="pipeline", typ="manual", path = "Dataframes"):
    f1s = []
    precs = []
    recalls = []
    overlaps = []
    
    num_manual = []
    stemmer = nltk.stem.PorterStemmer()
    sites = list(manual.keys())
    
    for site in sites:
        
        kws = manual[site]
        
        kws_hat = list(pd.read_csv(f"{path}/{typ}_{compare}_keywords_{site}.csv").keyword)
        
        kws = [stemmer.stem(kw) for kw in kws]

        kws_hat = [stemmer.stem(str(kw_hat)) for kw_hat in kws_hat][:max(len(kws), 10)]
        
        overlap = summary.number_of_overlaps(set(kws), set(kws_hat))
        
        recall = overlap / max(1, len(kws_hat))

        precision = overlap / max(1, len(kws))

        f1 =  2 * (precision * recall) / max(1, precision + recall)
        
        f1s.append(f1)
        precs.append(precision)
        recalls.append(recall)
        overlaps.append(overlap)
        num_manual.append(len(kws))
        
    summary_frame = pd.DataFrame({'site_id': [int(site) for site in sites], 
                                  'f1': f1s, 'precision': precs,
                                  'recall': recalls, '#overlaps': overlaps,
                                 '#manual keywords': num_manual})
    
    return summary_frame
    

In [ ]:
display(evaluate_manual(manual, compare="pipeline", typ=f"standard").drop("site_id", axis=1).describe())

,f1,precision,recall,#overlaps,#manual keywords
count,25.000000,25.000000,25.000000,25.000000,25.000000
mean,0.265096,0.371653,0.286319,3.120000,9.080000
std,0.256500,0.287083,0.207450,2.350886,3.161223
min,0.000000,0.000000,0.000000,0.000000,4.000000
25%,0.040816,0.142857,0.100000,1.000000,7.000000
50%,0.163265,0.333333,0.285714,3.000000,8.000000
75%,0.400000,0.529412,0.400000,4.000000,10.000000
max,0.750000,1.000000,0.600000,9.000000,17.000000


In [ ]:
def show_keywords_combinations(combinations, prefix):
  for i in range(len(combinations)):
    print(f"Results for combination {i}: {combinations[i]}")

    print("Pipeline Results: ")
    display(evaluate_manual(manual, compare="pipeline", typ=f"{prefix}_{i}"))

In [ ]:
def describe_combinations(combinations, prefix):
  for i in range(len(combinations)):
    print(f"Results for combination {i}: {combinations[i]}")

    print("Pipeline Results: ")
    pipe = evaluate_manual(manual, compare="pipeline", typ=f"{prefix}_{i}").drop("site_id", axis=1).mean()


    base1 = evaluate_manual(manual, compare="baseline1", typ=f"{prefix}_{i}").drop("site_id", axis=1).mean()

    base2 = evaluate_manual(manual, compare="baseline2", typ=f"{prefix}_{i}").drop("site_id", axis=1).mean()

    base3 = evaluate_manual(manual, compare="baseline3", typ=f"{prefix}_{i}").drop("site_id", axis=1).mean()

    base4 = evaluate_manual(manual, compare="baseline4", typ=f"{prefix}_{i}").drop("site_id", axis=1).mean()

    display(pd.DataFrame({"pipeline":pipe, "baseline1": base1, "baseline2": base2, "baseline3": base3, "baseline4": base4}).transpose())

## Results

In [ ]:
describe_combinations(combinations, "combination")

Results for combination 0: (<function match_sentences_semantic_search at 0x7f8169dd6280>, <function text_rank_importance at 0x7f81e7c6aee0>, 0.7)
Pipeline Results: 


,f1,precision,recall,#overlaps,#manual keywords
pipeline,0.499307,0.607759,0.477696,5.20,9.08
baseline1,0.091987,0.150368,0.140381,1.36,9.08
baseline2,0.015260,0.040198,0.061540,0.40,9.08
baseline3,0.126694,0.197173,0.166728,1.80,9.08
baseline4,0.141563,0.226566,0.173232,1.92,9.08


Results for combination 1: (<function match_sentences_tfidf_weighted at 0x7f8169dd61f0>, <function text_rank_importance at 0x7f81e7c6aee0>, 0.7)
Pipeline Results: 


,f1,precision,recall,#overlaps,#manual keywords
pipeline,0.473290,0.590267,0.460363,4.96,9.08
baseline1,0.117746,0.180925,0.164208,1.60,9.08
baseline2,0.007651,0.021968,0.035048,0.20,9.08
baseline3,0.150678,0.231123,0.187726,2.08,9.08
baseline4,0.141563,0.226566,0.173232,1.92,9.08
